In [1]:
import numpy as np
import ndio.remote.neurodata as ND
from datetime import datetime

startTime = datetime.now()

nd = ND()

token = 'kasthuri2015_ramon_v4'
channel = 'synapses'
res = 3
pixel_dim = 0.024*0.024*0.030  #can get from LIMS

In [3]:
'''
There were 1,700 synapses at a density of one synapse per 1.13 μm3.

'''

import ndio.ramon as ramon

synapse_ids = nd.get_ramon_ids(token, channel, ramon_type=ramon.RAMONSynapse)

# A priori known bounds for cylinders.  Alternatively we could sweep over entire volume - this is more efficient.
# TODO:  assume that all synapses are inside cylinders, which we know to be true - should do with manual masking or a 
# RAMONId predicate query

xbox = [694,1794]; 
ybox = [1750, 2460];
zbox = [1004, 1379];

# These calls take about 60 seconds to execute
gcyl = nd.get_volume('kat11greencylinder','annotation', xbox[0], xbox[1], ybox[0], ybox[1], zbox[0], zbox[1], resolution = res)
rcyl = nd.get_volume('kat11redcylinder','annotation', xbox[0], xbox[1], ybox[0], ybox[1], zbox[0], zbox[1], resolution = res)
bcyl = nd.get_volume('kat11mojocylinder','annotation', xbox[0], xbox[1], ybox[0], ybox[1], zbox[0], zbox[1], resolution = res)


mask = (gcyl.cutout + rcyl.cutout + bcyl.cutout) > 0

mask_pixels = sum(sum(sum(mask)))
mask_volume = mask_pixels * pixel_dim
print 'Mask Pixels: ' + str(mask_pixels)
print 'Mask Volume: ' + str(mask_volume) + ' um^3'

syn = nd.get_volume(token,channel, xbox[0], xbox[1], ybox[0], ybox[1], zbox[0], zbox[1], resolution = res)
syn_volume = np.count_nonzero(syn.cutout)

print 'density of synapses is: {} / um^3'.format(round(1.0*(np.shape(synapse_ids)[0]) / mask_volume,4))
print 'fraction of volume that is psd: ' + str(round(1.0*syn_volume/mask_pixels,4))

print datetime.now() - startTime

Mask Pixels: 90152653
Mask Volume: 1557.83784384 um^3
density of synapses is: 1.0913 / um^3
fraction of volume that is psd: 0.0089
0:03:54.708588


In [22]:
# Retrieve metadata for all objects
import ndio.ramon as ramon

nd = ND()
synapse_ids = nd.get_ramon_ids(token, channel, ramon_type=ramon.RAMONSynapse)
import copy
# Get all synapses
synAll = []
for x in range(np.shape(synapse_ids)[0]):
    s = nd.get_ramon_metadata(token,channel,[synapse_ids[x]])
    synAll.append(copy.deepcopy(s)[0])
#print '\ntime elapsed so far: ' + str(time.time()-start)


NameError: name 'time' is not defined

In [30]:
vars(synAll[3]) 

{'_status': 0,
 'author': 'unspecified',
 'confidence': 1.0,
 'cutout': None,
 'id': '4',
 'kvpairs': {'axonSynapseType': 'en-passant',
  'multiSynapseBouton': 'yes',
  'psdSize': '1393',
  'synapseLocation': 'spine',
  'vesicleCount': '281'},
 'resolution': 0,
 'segments': array([[1822,    2],
        [3832,    1]], dtype=uint32),
 'synapse_type': 0,
 'voxels': None,
 'weight': 0.0,
 'xyz_offset': (0, 0, 0)}

In [24]:
print synAll[3].kvpairs
print synAll[3].segments

{'multiSynapseBouton': 'yes', 'vesicleCount': '281', 'psdSize': '1393', 'axonSynapseType': 'en-passant', 'synapseLocation': 'spine'}
[[1822    2]
 [3832    1]]


In [31]:
vars(synAll[1])

{'_status': 0,
 'author': 'unspecified',
 'confidence': 1.0,
 'cutout': None,
 'id': '2',
 'kvpairs': {'axonSynapseType': 'en-passant',
  'multiSynapseBouton': 'no',
  'psdSize': '246',
  'synapseLocation': 'spine',
  'vesicleCount': '137'},
 'resolution': 0,
 'segments': array([[3422,    2],
        [3837,    1]], dtype=uint32),
 'synapse_type': 0,
 'voxels': None,
 'weight': 0.0,
 'xyz_offset': (0, 0, 0)}